In [ ]:
import requests
import json
import pandas as pd
from time import sleep

In [ ]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Emilia Romagna
df_er = df_comuni[df_comuni['cod_regione'] == '08']

In [ ]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

In [ ]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [ ]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
   print(f"\nProcessing record {idx + 1}")
   print(f"URL: {url}")

   response = requests.get(url, headers=headers)
   print(f"Response status code: {response.status_code}")
   
   data = response.json()

   comune = df_er.iloc[idx]['desc']
   provincia = df_er.iloc[idx]['provincia']
   regione = df_er.iloc[idx]['regione']
   url = df_er.iloc[idx]['url']

   print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

   # Start with basic comune info
   comune_info = {
       'comune': comune,
       'provincia': provincia,
       'regione': regione,
       'url': url
   }
   
   # Add candidate totals
   if 'cand' in data:
       for cand in data['cand']:
           nome_completo = cand['nome'] + " " + cand['cogn']
           comune_info[f"{nome_completo}_total"] = cand['voti']
           comune_info[f"{nome_completo}_perc"] = cand['perc']
           print(f"{nome_completo}: {cand['voti']} votes ({cand['perc']}%)")

   comuni_data.append(comune_info)
   
   print(f"Processed {comune}")
   sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())

In [ ]:
df_data

In [ ]:
# df_data['perc_diff'] = round(((df_data['MICHELE DE PASCALE_total'] - df_data['ELENA UGOLINI_total']) / df_data['ELENA UGOLINI_total']) * 100, 2)

# Get all columns containing '_total'
total_columns = [col for col in df_data.columns if '_total' in col]

# Sum all vote totals for each comune
df_data['total_votes'] = df_data[total_columns].sum(axis=1)

# Calculate the spectrum considering all votes
df_data['vote_spectrum'] = round((df_data['MICHELE DE PASCALE_total'] / df_data['total_votes']) * 100, 2)

In [ ]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data.head(2)

In [ ]:
# Remove rows where 'comune' is either 'Sassofeltrio' or 'Montecopiolo'
df_data = df_data[~df_data['comune'].isin(['Sassofeltrio', 'Montecopiolo'])]

# Use .str.replace() to handle string replacement more safely
df_data['comune'] = df_data['comune'].str.replace("Forli'", "Forlì", regex=False)
df_data['comune'] = df_data['comune'].str.replace("Montescudo - Monte Colombo", "Montescudo-Monte Colombo", regex=False)

In [ ]:
df_data.to_csv('../output/risultati_er.csv', index=False, encoding='UTF-8-sig')